# 11. Modeling data to features engineering

## 11.1 Import the Libs

In [2]:
import pandas as pd
import requests
import lxml
from bs4 import BeautifulSoup
import time
import pyodbc
import numpy as np
from urllib.error import URLError, HTTPError
from urllib.request import Request, urlopen
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import matplotlib.pyplot as plt

# 11.2 Reading data From Database

In [8]:
# Function Select
def SQLSelect(query):
    conn = pyodbc.connect('Trusted_Connection=yes',
                      driver = '{ODBC Driver 17 for SQL Server}',
                      server = 'localhost',
                      database = 'MLS_Project')
    out = pd.read_sql_query(query,conn)
    return out

query_all_rankTeam = '''
    Select 
    * 
    from TableRankingStandardTeamNames
'''
query_all_MatchDetails = '''
    Select 
    * 
    from TableMatchDetails
'''

def SQLTruncate(TableName):
    conn = pyodbc.connect('Trusted_Connection=yes',
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = 'localhost',
                          database = 'MLS_Project')

    cursor = conn.cursor()

    cursor.execute(f'''
    
                   TRUNCATE TABLE {TableName}
                     
                   ''')
    conn.commit()
    cursor.close()

In [10]:
df_rankTeam = SQLSelect(query_all_rankTeam)
df_MatchDetails = SQLSelect(query_all_MatchDetails)

In [21]:
df_rankTeam

,Generalkey,Rk,TEAM_NAME,MP,W,D,L,GF,GA,GD,...,XGA,XGD,XGD_90,NOTES,CONFERENCE,SEASON,W_MP,D_MP,L_MP,GD_MP
0,1,1,Philadelphia-Union,23,14,5,4,44,20,24,...,23.5,11.4,0.57,? First Round,EastConf,2020,0.608696,0.217391,0.173913,1.043478
1,2,2,Toronto-FC,23,13,5,5,33,26,7,...,22.5,8.6,0.43,? First Round,EastConf,2020,0.565217,0.217391,0.217391,0.304348
2,3,3,Columbus-Crew,23,12,5,6,36,21,15,...,28.1,0.5,0.03,? First Round,EastConf,2020,0.521739,0.217391,0.260870,0.652174
3,4,4,Orlando-City,23,11,8,4,40,25,15,...,24.3,3.2,0.16,? First Round,EastConf,2020,0.478261,0.347826,0.173913,0.652174
4,5,5,New-York-Red-Bulls,23,12,3,8,37,25,12,...,26.7,6.7,0.33,? First Round,EastConf,2020,0.521739,0.130435,0.347826,0.521739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,186,8,Vancouver-Whitecaps-FC,34,13,8,13,54,67,-13,...,60.3,-7.5,-0.22,,WestConf,2018,0.382353,0.235294,0.382353,-0.382353
186,187,9,Houston-Dynamo,34,10,8,16,58,58,0,...,55.3,1.6,0.05,,WestConf,2018,0.294118,0.235294,0.470588,0.000000
187,188,10,Minnesota-United,34,11,3,20,49,71,-22,...,61.4,-13.2,-0.39,,WestConf,2018,0.323529,0.088235,0.588235,-0.647059
188,189,11,Colorado-Rapids,34,8,7,19,36,63,-27,...,61.4,-21.0,-0.62,,WestConf,2018,0.235294,0.205882,0.558824,-0.794118


In [ ]:
#Removing unecessary columns
columns_to_remove = ['CONFERENCE','XGA','XG','XGD','XGD_90','NOTES','CONFERENCE','Generalkey']
df_rankTeam = df_rankTeam.drop(columns=columns_to_remove)

In [32]:
df_rankTeam.rename(columns = {'TEAM_NAME':'TEAM_NAME_RT','Rk':'RK','SEASON':'SEASON_RT'}, inplace = True)
df_rankTeam

,RK,TEAM_NAME_RT,MP,W,D,L,GF,GA,GD,PTS,PTS_MP,SEASON_RT,W_MP,D_MP,L_MP,GD_MP
0,1,Philadelphia-Union,23,14,5,4,44,20,24,47,2.04,2020,0.608696,0.217391,0.173913,1.043478
1,2,Toronto-FC,23,13,5,5,33,26,7,44,1.91,2020,0.565217,0.217391,0.217391,0.304348
2,3,Columbus-Crew,23,12,5,6,36,21,15,41,1.78,2020,0.521739,0.217391,0.260870,0.652174
3,4,Orlando-City,23,11,8,4,40,25,15,41,1.78,2020,0.478261,0.347826,0.173913,0.652174
4,5,New-York-Red-Bulls,23,12,3,8,37,25,12,39,1.70,2020,0.521739,0.130435,0.347826,0.521739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,8,Vancouver-Whitecaps-FC,34,13,8,13,54,67,-13,47,1.38,2018,0.382353,0.235294,0.382353,-0.382353
186,9,Houston-Dynamo,34,10,8,16,58,58,0,38,1.12,2018,0.294118,0.235294,0.470588,0.000000
187,10,Minnesota-United,34,11,3,20,49,71,-22,36,1.06,2018,0.323529,0.088235,0.588235,-0.647059
188,11,Colorado-Rapids,34,8,7,19,36,63,-27,31,0.91,2018,0.235294,0.205882,0.558824,-0.794118


# 11.3 Creating Features

In [13]:
#Add new column to df_MatchDetails
df_MatchDetails['SEASON_BEFORE'] = df_MatchDetails['SEASON']-1

In [14]:
df_MatchDetails

,Generalkey,GAME_Date,GAME_Time,WEEK_DAY,TEAM_AWAY,TEAM_HOME,RESULT_GAME,SEASON,SEASON_BEFORE
0,1,2020-02-29,19:30:00,Sat,Sporting-KC,Vancouver-Whitecaps-FC,A,2020,2019
1,2,2020-03-07,19:30:00,Sat,Houston-Dynamo,Sporting-KC,H,2020,2019
2,3,2020-08-21,18:30:00,Fri,Sporting-KC,Minnesota-United,A,2020,2019
3,4,2020-08-25,19:30:00,Tue,Houston-Dynamo,Sporting-KC,A,2020,2019
4,5,2020-08-29,19:00:00,Sat,Sporting-KC,Colorado-Rapids,D,2020,2019
...,...,...,...,...,...,...,...,...,...
3535,3536,2018-03-17,15:30:00,Sat,Orlando-City,New-York-Red-Bulls,H,2018,2017
3536,3537,2018-03-31,13:00:00,Sat,New-York-City-FC,Orlando-City,H,2018,2017
3537,3538,2018-06-02,19:30:00,Sat,Orlando-City,New-York-Red-Bulls,H,2018,2017
3538,3539,2018-07-26,20:00:00,Thu,New-York-Red-Bulls,Orlando-City,A,2018,2017


In [33]:
#Add new column to df_rankTeam
df_rankTeam['W_MP']  = df_rankTeam['W']/df_rankTeam['MP']
df_rankTeam['D_MP']  = df_rankTeam['D']/df_rankTeam['MP']
df_rankTeam['L_MP']  = df_rankTeam['L']/df_rankTeam['MP']
df_rankTeam['GD_MP'] = df_rankTeam['GD']/df_rankTeam['MP']
df_rankTeam

,RK,TEAM_NAME_RT,MP,W,D,L,GF,GA,GD,PTS,PTS_MP,SEASON_RT,W_MP,D_MP,L_MP,GD_MP
0,1,Philadelphia-Union,23,14,5,4,44,20,24,47,2.04,2020,0.608696,0.217391,0.173913,1.043478
1,2,Toronto-FC,23,13,5,5,33,26,7,44,1.91,2020,0.565217,0.217391,0.217391,0.304348
2,3,Columbus-Crew,23,12,5,6,36,21,15,41,1.78,2020,0.521739,0.217391,0.260870,0.652174
3,4,Orlando-City,23,11,8,4,40,25,15,41,1.78,2020,0.478261,0.347826,0.173913,0.652174
4,5,New-York-Red-Bulls,23,12,3,8,37,25,12,39,1.70,2020,0.521739,0.130435,0.347826,0.521739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,8,Vancouver-Whitecaps-FC,34,13,8,13,54,67,-13,47,1.38,2018,0.382353,0.235294,0.382353,-0.382353
186,9,Houston-Dynamo,34,10,8,16,58,58,0,38,1.12,2018,0.294118,0.235294,0.470588,0.000000
187,10,Minnesota-United,34,11,3,20,49,71,-22,36,1.06,2018,0.323529,0.088235,0.588235,-0.647059
188,11,Colorado-Rapids,34,8,7,19,36,63,-27,31,0.91,2018,0.235294,0.205882,0.558824,-0.794118


In [ ]:
#Merge new columns to df_rankTeam

In [54]:
# Adding HOME team details from last season
df_MatchDetailsV02 = pd.merge(df_MatchDetails,df_rankTeam, how='left',
                              right_on=["TEAM_NAME_RT", "SEASON_RT"], 
                              left_on=["TEAM_HOME", "SEASON_BEFORE"], 
                              suffixes=('_MATCH','_HOME'))

In [55]:
# Rename Specific Columns to HOME
df_MatchDetailsV02.rename(columns = {'RK':'HOME_SB_RK',
                                     'PTS_MP':'HOME_SB_PTS_MP', 
                                     'W_MP':'HOME_SB_W_MP',
                                     'D_MP':'HOME_SB_D_MP',
                                     'L_MP':'HOME_SB_L_MP',
                                     'GD_MP':'HOME_SB_GD_MP',
                                    },
                          inplace = True)
df_MatchDetailsV02.columns

Index(['Generalkey', 'GAME_Date', 'GAME_Time', 'WEEK_DAY', 'TEAM_AWAY',
       'TEAM_HOME', 'RESULT_GAME', 'SEASON', 'SEASON_BEFORE', 'HOME_SB_RK',
       'TEAM_NAME_RT', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'PTS',
       'HOME_SB_PTS_MP', 'SEASON_RT', 'HOME_SB_W_MP', 'HOME_SB_D_MP',
       'HOME_SB_L_MP', 'HOME_SB_GD_MP'],
      dtype='object')

In [56]:
#Removing unecessary columns
columns_to_remove = ['Generalkey','TEAM_NAME_RT', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'PTS']
df_MatchDetailsV02 = df_MatchDetailsV02.drop(columns=columns_to_remove)
df_MatchDetailsV02

,Generalkey,GAME_Date,GAME_Time,WEEK_DAY,TEAM_AWAY,TEAM_HOME,RESULT_GAME,SEASON,SEASON_BEFORE,HOME_SB_RK,HOME_SB_PTS_MP,SEASON_RT,HOME_SB_W_MP,HOME_SB_D_MP,HOME_SB_L_MP,HOME_SB_GD_MP
0,1,2020-02-29,19:30:00,Sat,Sporting-KC,Vancouver-Whitecaps-FC,A,2020,2019,12.0,1.00,2019.0,0.235294,0.294118,0.470588,-0.647059
1,2,2020-03-07,19:30:00,Sat,Houston-Dynamo,Sporting-KC,H,2020,2019,11.0,1.12,2019.0,0.294118,0.235294,0.470588,-0.529412
2,3,2020-08-21,18:30:00,Fri,Sporting-KC,Minnesota-United,A,2020,2019,4.0,1.56,2019.0,0.441176,0.235294,0.323529,0.264706
3,4,2020-08-25,19:30:00,Tue,Houston-Dynamo,Sporting-KC,A,2020,2019,11.0,1.12,2019.0,0.294118,0.235294,0.470588,-0.529412
4,5,2020-08-29,19:00:00,Sat,Sporting-KC,Colorado-Rapids,D,2020,2019,9.0,1.24,2019.0,0.352941,0.176471,0.470588,-0.147059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3535,3536,2018-03-17,15:30:00,Sat,Orlando-City,New-York-Red-Bulls,H,2018,2017,2.0,1.68,2017.0,0.470588,0.264706,0.264706,0.382353
3536,3537,2018-03-31,13:00:00,Sat,New-York-City-FC,Orlando-City,H,2018,2017,10.0,1.15,2017.0,0.294118,0.264706,0.441176,-0.558824
3537,3538,2018-06-02,19:30:00,Sat,Orlando-City,New-York-Red-Bulls,H,2018,2017,2.0,1.68,2017.0,0.470588,0.264706,0.264706,0.382353
3538,3539,2018-07-26,20:00:00,Thu,New-York-Red-Bulls,Orlando-City,A,2018,2017,10.0,1.15,2017.0,0.294118,0.264706,0.441176,-0.558824


In [58]:
# Adding AWAY team details from last season
df_MatchDetailsV03 = pd.merge(df_MatchDetailsV02,df_rankTeam, how='left',
                              right_on=["TEAM_NAME_RT", "SEASON_RT"], 
                              left_on=["TEAM_AWAY", "SEASON_BEFORE"], 
                              suffixes=('_MATCH','_AWAY'))
df_MatchDetailsV03

,Generalkey,GAME_Date,GAME_Time,WEEK_DAY,TEAM_AWAY,TEAM_HOME,RESULT_GAME,SEASON,SEASON_BEFORE,HOME_SB_RK,...,GF,GA,GD,PTS,PTS_MP,SEASON_RT_AWAY,W_MP,D_MP,L_MP,GD_MP
0,1,2020-02-29,19:30:00,Sat,Sporting-KC,Vancouver-Whitecaps-FC,A,2020,2019,12.0,...,49.0,67.0,-18.0,38.0,1.12,2019.0,0.294118,0.235294,0.470588,-0.529412
1,2,2020-03-07,19:30:00,Sat,Houston-Dynamo,Sporting-KC,H,2020,2019,11.0,...,49.0,59.0,-10.0,40.0,1.18,2019.0,0.352941,0.117647,0.529412,-0.294118
2,3,2020-08-21,18:30:00,Fri,Sporting-KC,Minnesota-United,A,2020,2019,4.0,...,49.0,67.0,-18.0,38.0,1.12,2019.0,0.294118,0.235294,0.470588,-0.529412
3,4,2020-08-25,19:30:00,Tue,Houston-Dynamo,Sporting-KC,A,2020,2019,11.0,...,49.0,59.0,-10.0,40.0,1.18,2019.0,0.352941,0.117647,0.529412,-0.294118
4,5,2020-08-29,19:00:00,Sat,Sporting-KC,Colorado-Rapids,D,2020,2019,9.0,...,49.0,67.0,-18.0,38.0,1.12,2019.0,0.294118,0.235294,0.470588,-0.529412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3535,3536,2018-03-17,15:30:00,Sat,Orlando-City,New-York-Red-Bulls,H,2018,2017,2.0,...,39.0,58.0,-19.0,39.0,1.15,2017.0,0.294118,0.264706,0.441176,-0.558824
3536,3537,2018-03-31,13:00:00,Sat,New-York-City-FC,Orlando-City,H,2018,2017,10.0,...,53.0,47.0,6.0,50.0,1.47,2017.0,0.411765,0.235294,0.352941,0.176471
3537,3538,2018-06-02,19:30:00,Sat,Orlando-City,New-York-Red-Bulls,H,2018,2017,2.0,...,39.0,58.0,-19.0,39.0,1.15,2017.0,0.294118,0.264706,0.441176,-0.558824
3538,3539,2018-07-26,20:00:00,Thu,New-York-Red-Bulls,Orlando-City,A,2018,2017,10.0,...,56.0,43.0,13.0,57.0,1.68,2017.0,0.470588,0.264706,0.264706,0.382353


In [59]:
# Rename Specific Columns to HOME
df_MatchDetailsV03.rename(columns = {'RK':'AWAY_SB_RK',
                                     'PTS_MP':'AWAY_SB_PTS_MP', 
                                     'W_MP':'AWAY_SB_W_MP',
                                     'D_MP':'AWAY_SB_D_MP',
                                     'L_MP':'AWAY_SB_L_MP',
                                     'GD_MP':'AWAY_SB_GD_MP',
                                    },
                          inplace = True)
df_MatchDetailsV03.columns

Index(['Generalkey', 'GAME_Date', 'GAME_Time', 'WEEK_DAY', 'TEAM_AWAY',
       'TEAM_HOME', 'RESULT_GAME', 'SEASON', 'SEASON_BEFORE', 'HOME_SB_RK',
       'HOME_SB_PTS_MP', 'SEASON_RT_MATCH', 'HOME_SB_W_MP', 'HOME_SB_D_MP',
       'HOME_SB_L_MP', 'HOME_SB_GD_MP', 'AWAY_SB_RK', 'TEAM_NAME_RT', 'MP',
       'W', 'D', 'L', 'GF', 'GA', 'GD', 'PTS', 'AWAY_SB_PTS_MP',
       'SEASON_RT_AWAY', 'AWAY_SB_W_MP', 'AWAY_SB_D_MP', 'AWAY_SB_L_MP',
       'AWAY_SB_GD_MP'],
      dtype='object')

In [60]:
#Removing unecessary columns
columns_to_remove = ['SEASON_RT_AWAY','SEASON_RT_MATCH','TEAM_NAME_RT', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'PTS']
df_MatchDetailsV03 = df_MatchDetailsV03.drop(columns=columns_to_remove)
df_MatchDetailsV03

,Generalkey,GAME_Date,GAME_Time,WEEK_DAY,TEAM_AWAY,TEAM_HOME,RESULT_GAME,SEASON,SEASON_BEFORE,HOME_SB_RK,...,HOME_SB_D_MP,HOME_SB_L_MP,HOME_SB_GD_MP,AWAY_SB_RK,AWAY_SB_PTS_MP,SEASON_RT_AWAY,AWAY_SB_W_MP,AWAY_SB_D_MP,AWAY_SB_L_MP,AWAY_SB_GD_MP
0,1,2020-02-29,19:30:00,Sat,Sporting-KC,Vancouver-Whitecaps-FC,A,2020,2019,12.0,...,0.294118,0.470588,-0.647059,11.0,1.12,2019.0,0.294118,0.235294,0.470588,-0.529412
1,2,2020-03-07,19:30:00,Sat,Houston-Dynamo,Sporting-KC,H,2020,2019,11.0,...,0.235294,0.470588,-0.529412,10.0,1.18,2019.0,0.352941,0.117647,0.529412,-0.294118
2,3,2020-08-21,18:30:00,Fri,Sporting-KC,Minnesota-United,A,2020,2019,4.0,...,0.235294,0.323529,0.264706,11.0,1.12,2019.0,0.294118,0.235294,0.470588,-0.529412
3,4,2020-08-25,19:30:00,Tue,Houston-Dynamo,Sporting-KC,A,2020,2019,11.0,...,0.235294,0.470588,-0.529412,10.0,1.18,2019.0,0.352941,0.117647,0.529412,-0.294118
4,5,2020-08-29,19:00:00,Sat,Sporting-KC,Colorado-Rapids,D,2020,2019,9.0,...,0.176471,0.470588,-0.147059,11.0,1.12,2019.0,0.294118,0.235294,0.470588,-0.529412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3535,3536,2018-03-17,15:30:00,Sat,Orlando-City,New-York-Red-Bulls,H,2018,2017,2.0,...,0.264706,0.264706,0.382353,10.0,1.15,2017.0,0.294118,0.264706,0.441176,-0.558824
3536,3537,2018-03-31,13:00:00,Sat,New-York-City-FC,Orlando-City,H,2018,2017,10.0,...,0.264706,0.441176,-0.558824,6.0,1.47,2017.0,0.411765,0.235294,0.352941,0.176471
3537,3538,2018-06-02,19:30:00,Sat,Orlando-City,New-York-Red-Bulls,H,2018,2017,2.0,...,0.264706,0.264706,0.382353,10.0,1.15,2017.0,0.294118,0.264706,0.441176,-0.558824
3538,3539,2018-07-26,20:00:00,Thu,New-York-Red-Bulls,Orlando-City,A,2018,2017,10.0,...,0.264706,0.441176,-0.558824,2.0,1.68,2017.0,0.470588,0.264706,0.264706,0.382353


In [68]:
df_MatchDetailsV03.columns

Index(['GAME_Date', 'GAME_Time', 'WEEK_DAY', 'TEAM_AWAY', 'TEAM_HOME',
       'RESULT_GAME', 'SEASON', 'SEASON_BEFORE', 'HOME_SB_RK',
       'HOME_SB_PTS_MP', 'HOME_SB_W_MP', 'HOME_SB_D_MP', 'HOME_SB_L_MP',
       'HOME_SB_GD_MP', 'AWAY_SB_RK', 'AWAY_SB_PTS_MP', 'AWAY_SB_W_MP',
       'AWAY_SB_D_MP', 'AWAY_SB_L_MP', 'AWAY_SB_GD_MP'],
      dtype='object')

In [66]:
columns_to_remove = ['SEASON_RT_AWAY']
df_MatchDetailsV03 = df_MatchDetailsV03.drop(columns=columns_to_remove)
df_MatchDetailsV03

,GAME_Date,GAME_Time,WEEK_DAY,TEAM_AWAY,TEAM_HOME,RESULT_GAME,SEASON,SEASON_BEFORE,HOME_SB_RK,HOME_SB_PTS_MP,HOME_SB_W_MP,HOME_SB_D_MP,HOME_SB_L_MP,HOME_SB_GD_MP,AWAY_SB_RK,AWAY_SB_PTS_MP,AWAY_SB_W_MP,AWAY_SB_D_MP,AWAY_SB_L_MP,AWAY_SB_GD_MP
0,2020-02-29,19:30:00,Sat,Sporting-KC,Vancouver-Whitecaps-FC,A,2020,2019,12.0,1.00,0.235294,0.294118,0.470588,-0.647059,11.0,1.12,0.294118,0.235294,0.470588,-0.529412
1,2020-03-07,19:30:00,Sat,Houston-Dynamo,Sporting-KC,H,2020,2019,11.0,1.12,0.294118,0.235294,0.470588,-0.529412,10.0,1.18,0.352941,0.117647,0.529412,-0.294118
2,2020-08-21,18:30:00,Fri,Sporting-KC,Minnesota-United,A,2020,2019,4.0,1.56,0.441176,0.235294,0.323529,0.264706,11.0,1.12,0.294118,0.235294,0.470588,-0.529412
3,2020-08-25,19:30:00,Tue,Houston-Dynamo,Sporting-KC,A,2020,2019,11.0,1.12,0.294118,0.235294,0.470588,-0.529412,10.0,1.18,0.352941,0.117647,0.529412,-0.294118
4,2020-08-29,19:00:00,Sat,Sporting-KC,Colorado-Rapids,D,2020,2019,9.0,1.24,0.352941,0.176471,0.470588,-0.147059,11.0,1.12,0.294118,0.235294,0.470588,-0.529412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3535,2018-03-17,15:30:00,Sat,Orlando-City,New-York-Red-Bulls,H,2018,2017,2.0,1.68,0.470588,0.264706,0.264706,0.382353,10.0,1.15,0.294118,0.264706,0.441176,-0.558824
3536,2018-03-31,13:00:00,Sat,New-York-City-FC,Orlando-City,H,2018,2017,10.0,1.15,0.294118,0.264706,0.441176,-0.558824,6.0,1.47,0.411765,0.235294,0.352941,0.176471
3537,2018-06-02,19:30:00,Sat,Orlando-City,New-York-Red-Bulls,H,2018,2017,2.0,1.68,0.470588,0.264706,0.264706,0.382353,10.0,1.15,0.294118,0.264706,0.441176,-0.558824
3538,2018-07-26,20:00:00,Thu,New-York-Red-Bulls,Orlando-City,A,2018,2017,10.0,1.15,0.294118,0.264706,0.441176,-0.558824,2.0,1.68,0.470588,0.264706,0.264706,0.382353


## 11.4 Inserting table to the Database

In [77]:
# Create a function to INSERT Table
def SQLInsertTable(TableGameDetails):
    base = pd.DataFrame(columns=['GAME_Date', 'GAME_Time', 'WEEK_DAY', 'TEAM_AWAY',
                                 'TEAM_HOME', 'RESULT_GAME', 'SEASON', 'SEASON_BEFORE', 'HOME_SB_RK',
                                 'HOME_SB_PTS_MP', 'HOME_SB_W_MP', 'HOME_SB_D_MP',
                                 'HOME_SB_L_MP', 'HOME_SB_GD_MP', 'AWAY_SB_RK', 'AWAY_SB_PTS_MP',
                                 'AWAY_SB_W_MP', 'AWAY_SB_D_MP', 'AWAY_SB_L_MP',
                                 'AWAY_SB_GD_MP'])
    
    TableBase = base.append(TableGameDetails).fillna('')

    conn = pyodbc.connect('Trusted_Connection=yes',
                          driver = '{ODBC Driver 17 for SQL Server}',
                          server = 'localhost',
                          database = 'MLS_Project')

    cursor = conn.cursor()

    for index, row in TableBase.iterrows():

        cursor.execute('''

            INSERT INTO TableMatchDetailsFull ( 
                GAME_Date,
                GAME_Time,
                WEEK_DAY,
                
                TEAM_AWAY,
                TEAM_HOME,
                RESULT_GAME,
                
                SEASON,
                SEASON_BEFORE,
                
                HOME_SB_RK,
                HOME_SB_PTS_MP,
                HOME_SB_W_MP, 
                HOME_SB_D_MP,
                HOME_SB_L_MP,
                HOME_SB_GD_MP,
                
                AWAY_SB_RK,
                AWAY_SB_PTS_MP,
                AWAY_SB_W_MP,
                AWAY_SB_D_MP,
                AWAY_SB_L_MP,
                AWAY_SB_GD_MP
                
                )
            Values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', # mesma qtde de variaveis

            row['GAME_Date'],
            row['GAME_Time'],
            row['WEEK_DAY'],
                       
            row['TEAM_AWAY'],
            row['TEAM_HOME'],
            row['RESULT_GAME'],
                       
            row['SEASON'],
            row['SEASON_BEFORE'],
                       
            row['HOME_SB_RK'],
            row['HOME_SB_PTS_MP'],
            row['HOME_SB_W_MP'],
            row['HOME_SB_D_MP'],
            row['HOME_SB_L_MP'],
            row['HOME_SB_GD_MP'],
                       
            row['AWAY_SB_RK'],
            row['AWAY_SB_PTS_MP'],
            row['AWAY_SB_W_MP'],
            row['AWAY_SB_D_MP'],
            row['AWAY_SB_L_MP'],
            row['AWAY_SB_GD_MP']
        )

    conn.commit()
    cursor.close()

In [78]:
SQLInsertTable(df_MatchDetailsV03)